In [1]:
import gzip
import subprocess
import os,sys,csv,re
from optparse import OptionParser,OptionGroup
import pandas as pd

# Umitools
import pyximport
pyximport.install(build_in_temp=False)
from umi_tools._dedup_umi import edit_distance

In [ ]:
path = 'Your path to fastq file here'
ref  = 'Your path to reference file here' # bwa index first
samples =[s for s in os.listdir(path) if '.gz' in s and 'R1' in s]
for s in samples:
    cmd = 'bwa mem ' + ref + ' ' + path+s + \
    '| samtools view -F 0x904 | awk \'$3 == "Hek3" {{print $1"\\t"$3"\\t"$10; next}}\'> ' + path  + s.split('_S')[0] + '.txt&'
    p=subprocess.Popen(cmd,stdout=subprocess.DEVNULL,stderr=subprocess.STDOUT, shell=True);
    p.wait();

In [ ]:
## Wait until all alignment is done
samples =sorted([s.split('.txt')[0] for s in os.listdir(path) if '.txt' in s]) 
for s in samples:
    cmd = "less "+ path+s + ".txt | awk \' match($3, /GCACG([ATCG]{5})TGATG/) {print substr($3, RSTART+5,RLENGTH-10)}\' | sort -k1n | uniq -c | sort -k1nr |awk '{print $2\"\\t\"$1}'  > "+\
    path + s.split('_S')[0] + "_bc_count.tsv &"
    p=subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True);
    p.wait();

In [3]:
# Wait until all regex is done
os.chdir(path)
cmd = "for i in \`ls -1 *.txt\`; do less $i | echo $i $((\`wc -l\`)) | cat >> Read_count.csv & done; "
p=subprocess.Popen(cmd,stdout=subprocess.DEVNULL,stderr=subprocess.STDOUT, shell=True);
p.wait();

In [ ]:
def str2array(s):
    a,b= s.split('\t')[:2]
    return (a,int(b))

def merge_edit_1(counts):
    keep,collapse = [],counts[0:]
    while len(collapse)>0:
        ref = list(collapse[0]) # need to convert to list here otherwise will keep updating the value to new_cluster
        temp = []
        for s in collapse[1:]:
            s1 = ref[0]
            s2 = s[0]
            if abs(len(s1)-len(s2))<=1 and edit_distance(s1.encode('utf-8'),s2.encode('utf-8'))<2:
                ref[-1] += s[-1]
            else:
                temp.append(s)
        keep.append(ref)
        collapse = temp[0:]
    return np.array(keep,dtype=object)

def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

import re
def natural_key(string_):
    """See http://www.codinghorror.com/blog/archives/001018.html"""
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]


def reverse_complement(seq):
    """This function returns the reverse_complement sequence of the input sequence
    from 3' to 5' """
    complement = {'A':'T', 'C':'G','G':'C', 'T':'A', 'N':'N'}
    rcomp_seq = ''
    for base in seq:
        rcomp_seq = complement[base] + rcomp_seq   
    return rcomp_seq

In [ ]:
path = 'Same to the path above, where you keep all the fastq.gz and parsed files'
samples =sorted([s.split('.txt')[0] for s in os.listdir(path) if '.txt' in s])
read_count = pd.read_table(path+'Read_count.csv',header=None,names=['sample','counts'],sep=' ' )
edits = []
for s in samples: 
    file = path+s
    rcount = int(read_count[read_count['sample'].str.contains(s)]['counts'])
    with open(file +'_bc_count.tsv','r') as f:
        temp = f.readlines()
        test = sum(merge_edit_1([a for a in [str2array(s) for s in temp]])[:,1])
    f.close()
    edits.append([s,test/rcount*100])

In [ ]:
df = pd.DataFrame(edits,columns=['Sample','Editing efficiency'])